<table><tbody><tr><th><p><img alt="Emblema" src="https://upload.wikimedia.org/wikipedia/en/6/69/Seal_of_Pablo_de_Olavide_University.png" style="width:150px;"></p></th><th><p><strong>Inteligencia Artificial</strong></p><p><strong>Grado en Ingeniería Informática en Sistemas de Información</strong></p><p><strong>TRABAJO IA</strong></p><h1>Redes Neuronales</h1></th></tr></tbody></table>


In [ ]:
import pandas as pd
import numpy as np
import math 
import matplotlib.pyplot as plt
import seaborn as sns

### Análisis de los datos
El conjunto de datos está enfocado a la predicción del coste del seguro médico, y nos brinda 54 características con las que operar.

En ésta página web podremos ver en profundidad qué representan las distintas características.

En resumen, éstas pueden condensarse en 6 grupos:

1. Demográficas y Socioeconómicas.
2. Estilo de vida y hábitos.
3. Salud y datos clínicos.
4. Uso del seguro médico y procedimientos utilizados.
5. Seguro y Políticas.
6. Costes médicos y pagos.

Carguemos los datos y echémosles un vistazo:

In [ ]:
#Funcion para leer los datos
def read_csv(file_name):
    try:
        df = pd.read_csv(file_name)
        return df
    except FileNotFoundError:
        print(f"ERROR: El archivo '{file_name}' no se encuentra.")
        return None

In [ ]:
#Llamamos a la funcion para cargar nuestro csv personalizado
df = read_csv('medical_insurance.csv')
#Comprobamos que se ha cargado bien
df.head()

In [ ]:
# Quitar espacios al inicio y fin de los nombres de columna para no haber fallos en en la programacion 
df.columns = df.columns.str.strip()

## Variables categóricas
Las variables categóricas han de pasarse a numéricas. Para ello, podemos usar dos métodos:

Label encoding: Cambiamos cada tipo de valor por un número entero. Para valores que tengan relación y cierta continuidad, tendría sentido usar ésta opción, ya que implícitamente se crea una relación mayor-menor entre los valores. Ej.: Frecuencia de uso de alcohol -> ['Occasional' -> 1, 'Weekly' -> 2, 'Daily' -> 3]

One-hot encoding: Se crea, por cada valor único de la tabla, una nueva columna binaria. En ellas se pondría un 0 o un 1 dependiendo del valor de la fila original. Ésto, en contraste con el label encoding, no crea un orden intrínseco para los valores de la columna. Sería ideal, por ejemplo, para la columna región, que tiene como valores únicos norte, sur, este y oeste (no hay orden aquí, ej.: sur no es "menor" que norte).

Teniendo en cuenta ambos métodos, vamos a realizar la conversión:

In [ ]:
# Label encoding

# Tenemos que crear unos "mapas" mediante los cuales
# le asignaremos valores numéricos a los distintos
# valores de cada columna.

# La educación la ordenaremos de menor a mayor formación
education_map = {
    'No HS': 0, 'HS': 1, 'Some College': 2,
    'Bachelors': 3, 'Masters': 4, 'Doctorate': 5
}

# Ésta variable podría ser codificada mediante otras técnicas
# pero me ha parecido interesante repartirla así
smoker_map = {
    'Never': 0, 'Former': 1, 'Current': 2
}

# Ordenamos según la frecuencia
alcohol_map = {
    'None': 0, 'Occasional': 1, 'Weekly': 2, 'Daily': 3
}

# Ordenamos por rango, de peor a mejor.
tier_map = {
    'Bronze': 0, 'Silver': 1, 'Gold': 2, 'Platinum': 3
}

# Aplicamos los cambios
df['education_encoded'] = df['education'].map(education_map)
df['smoker_encoded'] = df['smoker'].map(smoker_map)
df['alcohol_freq_encoded'] = df['alcohol_freq'].map(alcohol_map)
df['network_tier_encoded'] = df['network_tier'].map(tier_map)

# get_dummies no borra las columnas originales, así que tenemos que hacerlo nosotros.
df.drop(columns=['education', 'smoker', 'alcohol_freq', 'network_tier'], inplace=True)

df.head()
print(df.columns)

In [ ]:
# One-Hot Encoding

# El one-hot es mucho más sencillo si hacemos uso de get_dummies,
# ya que lo hace automáticamente. Pero primero, tenemos que indicarle
# sobre qué columnas queremos el onehot:
nominal_cols = [
    'sex', 'region', 'urban_rural', 
    'marital_status', 'employment_status', 'plan_type'
]

# IMPORTANTE: Si no le ponemos el dtype=int, en vez de devolver 0 o 1, devolverá
# true o false. Para el modelo de regresión logística, que al fin y al cabo
# no son más que un conjunto de fórmulas matemáticas, necesitamos el
# valor numérico para poder operar.
df = pd.get_dummies(df, columns=nominal_cols, dtype=int)

# Vemos el resultado
df.head()

<h1>Correlacion</h1><hr>

In [ ]:
#Para poder aplicar este algoritmo, primero tenemos que